
# EDA de Noticias + Embeddings (multilingüe) + Agregaciones Diarias

Este notebook está pensado para analizar el corpus de noticias obtenido del *scraper de La Nación* (secciones **Política** y **Economía**), generar **embeddings** con un modelo de **Hugging Face** en español/bilingüe, y calcular **agregaciones diarias** (conteos y vectores promedio por día).

> **Requisitos**
> - Python 3.10+
> - `pandas`, `numpy`, `matplotlib`
> - `sentence-transformers` (para embeddings de oraciones)
>
> **Entrada esperada**: un archivo `.parquet` o `.csv` con al menos estas columnas:
> - `id`
> - `seccion`
> - `fecha_publicacion` (ISO 8601)
> - `titulo`
> - `url`
> - `bajada` (opcional)
> - `texto` (opcional)


## 1) Instalación

Segun requirements.txt

In [ ]:

# Ejecuta esta celda solo si te faltan dependencias
# (Comenta lo que ya tengas instalado)
# Nota: usa un entorno virtual (venv/conda) antes de instalar.

# %pip install --upgrade pip
%pip install pandas numpy matplotlib sentence-transformers pyarrow


## 2) Imports y configuración

In [ ]:

import os
import math
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Opciones de pandas para display
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.width", 140)

# Helper para plots (sin seaborn, solo matplotlib)
def lineplot(x, y, title=None, xlabel=None, ylabel=None):
    plt.figure(figsize=(10,4))
    plt.plot(x, y)
    if title: plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    if ylabel: plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()


def imshow_matrix(mat, xticks=None, yticks=None, title=None, xlabel=None, ylabel=None):
    plt.figure(figsize=(7,6))
    im = plt.imshow(mat, aspect='auto')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    if title: plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    if ylabel: plt.ylabel(ylabel)
    if xticks is not None:
        plt.xticks(range(len(xticks)), xticks, rotation=90)
    if yticks is not None:
        plt.yticks(range(len(yticks)), yticks)
    plt.tight_layout()
    plt.show()


def to_date_only(s):
    # Convierte ISO 8601 a fecha (YYYY-MM-DD)
    try:
        return pd.to_datetime(s, utc=True).date()
    except Exception:
        # Intentos alternativos
        try:
            return pd.to_datetime(s).date()
        except Exception:
            return pd.NaT


## 3) Cargar dataset de noticias

In [ ]:

# Ajusta la ruta al archivo generado por el scraper
# Ejemplo: noticias_LN_2025Q1.parquet
DATA_PATH = Path("./noticias_LN_2025Q1.parquet")

if DATA_PATH.suffix.lower() == ".parquet":
    df = pd.read_parquet(DATA_PATH)
else:
    df = pd.read_csv(DATA_PATH)

print("Dimensiones:", df.shape)
df.head(5)


## 4) Limpieza mínima y normalización de campos

In [ ]:

# Aseguramos columnas esperadas
expected_cols = ['id','seccion','fecha_publicacion','titulo','url']
for c in expected_cols:
    if c not in df.columns:
        raise ValueError(f"Falta la columna requerida: {c}")

# Texto base: preferimos `texto` si existe; si no, construimos a partir de `titulo` + `bajada`
if 'texto' in df.columns and df['texto'].notna().any():
    df['texto_base'] = df['texto'].fillna('')
else:
    df['texto_base'] = ''
    if 'titulo' in df.columns:
        df['texto_base'] = df['texto_base'] + df['titulo'].fillna('')
    if 'bajada' in df.columns:
        df['texto_base'] = (df['texto_base'] + ' ' + df['bajada'].fillna('')).str.strip()

# Normalizamos fecha a día
df['fecha_dia'] = df['fecha_publicacion'].apply(to_date_only)
df = df.dropna(subset=['fecha_dia'])

# Longitud de textos (caracteres y palabras aprox)
df['len_chars'] = df['texto_base'].fillna('').str.len()
df['len_words'] = df['texto_base'].fillna('').str.split().apply(len)

df[['id','seccion','fecha_dia','len_chars','len_words']].head(10)


## 5) EDA básico

In [ ]:

# Conteo por sección
conteo_seccion = df['seccion'].value_counts().sort_values(ascending=False)
display(conteo_seccion)

# Conteo diario
conteo_diario = df.groupby('fecha_dia')['id'].nunique().sort_index()
display(conteo_diario.head(10))

# Serie temporal de notas por día
lineplot(conteo_diario.index, conteo_diario.values, 
         title="Cantidad de noticias por día", xlabel="Fecha", ylabel="Notas/día")

# Distribución de longitudes (caracteres)
plt.figure(figsize=(8,4))
plt.hist(df['len_chars'], bins=40)
plt.title("Distribución de longitudes de texto (caracteres)")
plt.xlabel("Caracteres")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()


## 6) Embeddings (Hugging Face, multilingüe)

In [ ]:

# Elegí uno de los modelos multilingües recomendados:
# - 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'  (ligero y bueno)
# - 'distiluse-base-multilingual-cased-v1' (ligero y popular)

MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = SentenceTransformer(MODEL_NAME)

# Texto a embebeder
texts = df['texto_base'].fillna('').tolist()

# Batch encode (ajustá batch_size si es necesario)
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

emb_dim = embeddings.shape[1]
print("Embeddings shape:", embeddings.shape)


### 6.1) (Opcional) Guardar embeddings por noticia

In [ ]:

# Guardamos en un parquet separado (vector como lista)
out_news_emb = Path('./news_embeddings.parquet')
df_emb = pd.DataFrame({
    'id': df['id'].values,
    'fecha_dia': df['fecha_dia'].astype(str).values,
    'seccion': df['seccion'].values,
    'url': df['url'].values,
    'titulo': df.get('titulo', pd.Series(['']*len(df))).values,
    'embedding': [emb.tolist() for emb in embeddings]
})
df_emb.to_parquet(out_news_emb, index=False)
print(f"Guardado: {out_news_emb} ({len(df_emb)} filas)")


## 7) Agregación diaria de embeddings (promedio por día)

In [ ]:

# Construimos matriz día x embedding (promedio)
df['__row'] = range(len(df))  # índice para recuperar luego
df_day_groups = df.groupby('fecha_dia').indices  # dict: fecha -> indices de filas

fechas = sorted(list(df_day_groups.keys()))
day_vectors = []
day_counts = []

for fecha in fechas:
    idxs = df_day_groups[fecha]
    vecs = embeddings[idxs]
    day_vectors.append(vecs.mean(axis=0))
    day_counts.append(len(idxs))

day_vectors = np.vstack(day_vectors)  # shape: (n_dias, emb_dim)
day_counts = np.array(day_counts)

print("Matriz diaria:", day_vectors.shape)
print("Conteos por día (primeros 10):", day_counts[:10])


## 8) Visualizaciones sobre agregaciones diarias

In [ ]:

# PCA a 2D para visualizar evolución diaria
pca = PCA(n_components=2, random_state=42)
xy = pca.fit_transform(day_vectors)

plt.figure(figsize=(8,6))
plt.scatter(xy[:,0], xy[:,1], s=np.clip(day_counts, 10, 120))
for i, f in enumerate(fechas):
    plt.text(xy[i,0], xy[i,1], str(f), fontsize=8)

plt.title("Embeddings diarios (PCA 2D) - tamaño ~ cantidad de notas")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

# Similaridad coseno entre días
sim = cosine_similarity(day_vectors)
imshow_matrix(sim, xticks=[str(f) for f in fechas], yticks=[str(f) for f in fechas],
              title="Similaridad coseno entre días", xlabel="Días", ylabel="Días")


## 9) (Opcional) Embeddings por sección y día

In [ ]:

# Promedio por (fecha_dia, seccion)
pairs = df.groupby(['fecha_dia','seccion']).indices
pairs_keys = sorted(list(pairs.keys()))

pair_vectors = []
for k in pairs_keys:
    idxs = pairs[k]
    pair_vectors.append(embeddings[idxs].mean(axis=0))

pair_vectors = np.vstack(pair_vectors)

# Proyección PCA para ver si hay separación por secciones a lo largo del tiempo
xy2 = PCA(n_components=2, random_state=42).fit_transform(pair_vectors)

plt.figure(figsize=(8,6))
for (f, s), (x, y) in zip(pairs_keys, xy2):
    plt.scatter(x, y)
    plt.text(x, y, f"{f}-{s}", fontsize=8)
plt.title("Embeddings promedio por (día, sección)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()


## 10) Export de agregaciones diarias

In [ ]:

# Guardamos los vectores diarios y conteos
daily_df = pd.DataFrame({
    'fecha_dia': [str(f) for f in fechas],
    'count_noticias': day_counts,
    'embedding': [v.tolist() for v in day_vectors]
})
out_daily = Path('./daily_news_embeddings.parquet')
daily_df.to_parquet(out_daily, index=False)
print(f"Guardado: {out_daily} ({len(daily_df)} filas)")

# CSV con conteos diarios (sin embeddings)
conteo_diario = pd.Series(day_counts, index=pd.Index(fechas, name='fecha_dia'), name='count_noticias')
conteo_csv = Path('./daily_news_counts.csv')
conteo_diario.to_csv(conteo_csv, header=True)
print(f"Guardado: {conteo_csv}")



## 11) Checklist / Próximos pasos

- [ ] Añadir **detección de idioma** para métricas (es/en) por día (fasttext-langdetect o spacy-cld3).
- [ ] Probar otros modelos de embeddings (por ejemplo, `intfloat/multilingual-e5-base` con formato de prompt apropiado).
- [ ] Construir **índice vectorial** (FAISS) para búsquedas por similitud entre días o noticias.
- [ ] Agregar **topic modeling** (BERTopic o LDA clásico como línea base).
- [ ] Incorporar **sentimiento** por día/sección y correlacionarlo con los ejes PCA.
